## PROJETO PÁSCOA COM DADOS
##### Apoiei o Projeto Dado Humanizado do Prof. Cláudio Bonel. Foi uma iniciativa chamada "Páscoa com Dados", que entregou ovos de páscoa para 100 crianças do Complexo do Alemão/RJ em 2022. 

In [1]:
# Importando as bibliotecas necessarias
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

### COLETA DE DADOS DO SITE DO IBGE
##### Webscraping para o Data Warehouse. Criando uma dimensão em Python

In [2]:
#Coletando dados do portal do IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

In [4]:
# Coletando dados do RJ
#pd.read_html(url)[21] # 21 por que é a posicao no corpo da pagina do IBGE. É uma lista de tabelas 
Dados_IBGE = pd.read_html(url, match='Municípios do Rio de Janeiro')[0]
Dados_IBGE.head(10)

,Municípios do Rio de Janeiro,Códigos
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233
5,Arraial do Cabo,3300258
6,Barra Mansa,3300407
7,Barra do Piraí,3300308
8,Belford Roxo,3300456
9,Bom Jardim,3300506


In [5]:
type(Dados_IBGE)

pandas.core.frame.DataFrame

In [5]:
# Dimensao dos dados
Dados_IBGE.shape

(92, 2)

In [6]:
# Informacoes
Dados_IBGE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Municípios do Rio de Janeiro  92 non-null     object
 1   Códigos                       92 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [7]:
# Verificando a existencia de dados nulos
Dados_IBGE.isnull().sum()

Municípios do Rio de Janeiro    0
Códigos                         0
dtype: int64

In [8]:
# Ja estava como DataFrame. Caso fosse necessario, colocariamos como DataFrame conforme abaixo.
Dados_IBGE = pd.DataFrame(pd.read_html(url, match='Municípios do Rio de Janeiro')[0])
Dados_IBGE.head(10)

,Municípios do Rio de Janeiro,Códigos
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233
5,Arraial do Cabo,3300258
6,Barra Mansa,3300407
7,Barra do Piraí,3300308
8,Belford Roxo,3300456
9,Bom Jardim,3300506


In [9]:
# Tipo DataFrame
type(Dados_IBGE)

pandas.core.frame.DataFrame

In [10]:
# Renomenado as colunas do DataFrame
Dados_IBGE = Dados_IBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})
Dados_IBGE.head()

,nmMunic,codMunic
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233


In [11]:
# Alterando o nome do Index que o pandas cria automaticamente
# Ficou fora de ordem para diferenciar o que é index. O index fica embaixo
Dados_IBGE.index.name = 'idMunic'
Dados_IBGE.head()

,nmMunic,codMunic
idMunic,,
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233


In [12]:
# Alterando o index para inciar em 1 ao inves de 0. O pandas cria o index automaticamente a partir do 0
# Vai facilitar para inserir do Banco de Dados
Dados_IBGE.index = Dados_IBGE.index + 1
Dados_IBGE.head()

,nmMunic,codMunic
idMunic,,
1,Angra dos Reis,3300100
2,Aperibé,3300159
3,Araruama,3300209
4,Areal,3300225
5,Armação dos Búzios,3300233


In [13]:
# Adicionando uma coluna de data e hora da carga. Colocará a hora atual do sistema
Dados_IBGE['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')
Dados_IBGE.head()

,nmMunic,codMunic,dtCarga
idMunic,,,
1,Angra dos Reis,3300100,08/04/2022 08:55
2,Aperibé,3300159,08/04/2022 08:55
3,Araruama,3300209,08/04/2022 08:55
4,Areal,3300225,08/04/2022 08:55
5,Armação dos Búzios,3300233,08/04/2022 08:55


In [14]:
# Informacoes
Dados_IBGE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 1 to 92
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   nmMunic   92 non-null     object
 1   codMunic  92 non-null     int64 
 2   dtCarga   92 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.3+ KB


### CRIANDO O BANCO DE DADOS ODS

In [14]:
# Manipulando o sistema de arquivos no Windows
endereco = Path('E:\\Projetos\\Python\\08 - Prof Bonel\\Python Integracao Dados com SQL\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

if endereco.exists():
    if (BDODS.exists() and BDDW.exists()):
        print('Bancos de dados já existem!')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe! Favor, verificar!')
        

Bancos de dados já existem!


In [15]:
# Listando os arquivos na pasta
!ls

01 - WebScrapingDadosIBGEdoRJ.ipynb
pascoaDW.db
pascoaODS.db


### MANIPULANDO OS BANCOS DE DADOS CRIADOS
#### SQLite

In [16]:
# CONEXAO COM O BANCO DE DADOS SQLite
# Conectando no BDODS
conexaoODS = sql.connect(BDODS)

# Criando a tabela tbLogMunic e carregando os dados do DataFrame Dados_IBGE
# Sera uma tabela para armazenar o historico. A cada execucao fara append ao final da base de dados
Dados_IBGE.to_sql('tbLogMunic',conexaoODS,if_exists="append") # adicionara novos dados

#Confirmando a transacao
conexaoODS.commit()

#Fechando a conexao
conexaoODS.close()

print('Carga do BDODS concluída!')

Carga do BDODS concluída!


In [17]:
#Conectando no BDDW
conexaoDW = sql.connect(BDDW)

#Selecionando somente as colunas para criacao da tabela dMunicipio
Dados_IBGE = Dados_IBGE[['nmMunic','codMunic']]

#Criando a tabela dMunicipio e carregando os dados do DataFrame Dados_IBGE
# Sempre que for executado o script, sera feita a substituicao dos dados, para que fiquem atualizados
Dados_IBGE.to_sql('dMunicipio',conexaoDW,if_exists="replace")

#Confirmando a transacao
conexaoDW.commit()

#Fechando a conexao
conexaoDW.close()

print('Carga do DW concluída!')

Carga do DW concluída!
